In [6]:
# !pip install mlflow boto3 awscli optuna xgboost imbalanced-learn

In [7]:
import numpy as np
import pandas as pd
import mlflow

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sudhirjoon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sudhirjoon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
mlflow.set_tracking_uri("http://13.60.79.0:5000")
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

2024/12/29 16:18:02 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5 - ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://capstone-yt-mlflow-bucket/542391321365143307', creation_time=1735485482149, experiment_id='542391321365143307', last_update_time=1735485482149, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [3]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [11]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 2)  # Bigram setting
max_features = 1000  # Set max_features to 1000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

# Step 3: Imbalance dataset using ADASYN
adasyn = ADASYN(random_state=42)
X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)


# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Bigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
     
    # Log trial in MLflow
    with mlflow.start_run(nested=True):  # Nested allows Optuna trials to appear under a single experiment
        # Log trial number in MLflow
        mlflow.set_tag("mlflow.runName", f"XGB_Trial_{trial.number}")
        # Log trial parameters
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("max_depth", max_depth)

        # Train and evaluate the model
        model.fit(X_train_vec, y_train)
        y_pred = model.predict(X_test_vec)
        accuracy = accuracy_score(y_test, y_pred)

        # Log trial metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("trial_number", trial.number)

    # Return the metric for Optuna optimization
    return accuracy


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], 
                               learning_rate=best_params['learning_rate'], 
                               max_depth=best_params['max_depth'], 
                               random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()


[I 2024-12-29 16:46:59,798] A new study created in memory with name: no-name-3061657d-4cba-49eb-9fe8-342dba1eee7d
[I 2024-12-29 16:47:04,340] Trial 0 finished with value: 0.7798697870842864 and parameters: {'n_estimators': 292, 'learning_rate': 0.009258540776564714, 'max_depth': 5}. Best is trial 0 with value: 0.7798697870842864.


🏃 View run XGB_Trial_0 at: http://13.60.79.0:5000/#/experiments/542391321365143307/runs/8dcefc658465483d8706f650d473bfd4
🧪 View experiment at: http://13.60.79.0:5000/#/experiments/542391321365143307


[I 2024-12-29 16:47:06,380] Trial 1 finished with value: 0.7902516276614464 and parameters: {'n_estimators': 58, 'learning_rate': 0.023137998929889048, 'max_depth': 10}. Best is trial 1 with value: 0.7902516276614464.


🏃 View run XGB_Trial_1 at: http://13.60.79.0:5000/#/experiments/542391321365143307/runs/61ef687b55e84610869b575c85fc9838
🧪 View experiment at: http://13.60.79.0:5000/#/experiments/542391321365143307


[I 2024-12-29 16:47:11,487] Trial 2 finished with value: 0.70983635403836 and parameters: {'n_estimators': 216, 'learning_rate': 0.00016252862890169084, 'max_depth': 9}. Best is trial 1 with value: 0.7902516276614464.


...

🏃 View run XGB_Trial_28 at: http://13.60.79.0:5000/#/experiments/542391321365143307/runs/b1e4195aeea7414ebfe4c025e3a17443
🧪 View experiment at: http://13.60.79.0:5000/#/experiments/542391321365143307


[I 2024-12-29 16:48:51,949] Trial 29 finished with value: 0.7994017244413162 and parameters: {'n_estimators': 284, 'learning_rate': 0.013033862497713211, 'max_depth': 5}. Best is trial 15 with value: 0.8835122294562731.


🏃 View run XGB_Trial_29 at: http://13.60.79.0:5000/#/experiments/542391321365143307/runs/4d77f5e7d6a64141bc033a5e3c676e8e
🧪 View experiment at: http://13.60.79.0:5000/#/experiments/542391321365143307


2024/12/29 16:48:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBoost_ADASYN_TFIDF_Bigrams at: http://13.60.79.0:5000/#/experiments/542391321365143307/runs/484f652e22c14078822268957599bcd0
🧪 View experiment at: http://13.60.79.0:5000/#/experiments/542391321365143307


In [ ]:
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)
    return study.best_params  # Return best parameters

best_params = run_optuna_experiment()
print("Best parameters:", best_params)